# Testing EUGENE training

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing the training of EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

# Set-up

In [1]:
import numpy as np
import pandas as pd

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [2]:
from eugene.dataloading.SeqDataModule import SeqDataModule

# Model params

In [4]:
MODEL = "fcn"
STRAND = "ss"
TASK = "binary_classification"

# Instantiate model and data via config

In [5]:
import claim.utils as cu
from eugene.models.FCN import FCN
from eugene.models.CNN import CNN
from eugene.models.RNN import RNN
from eugene.models.hybrid import hybrid

In [6]:
import pytorch_lightning as pl
from pytorch_lightning.utilities.cli import LightningArgumentParser
from pytorch_lightning.loggers import TensorBoardLogger

In [7]:
parser = LightningArgumentParser()
parser.add_lightning_class_args(pl.Trainer, nested_key="trainer")
parser.add_lightning_class_args(SeqDataModule, nested_key="data")
parser.add_lightning_class_args(FCN, nested_key="model");

In [8]:
config = "/cellar/users/aklie/projects/EUGENE/config/full/1_full_fcn.yaml"
yml = parser.parse_path(cfg_path=config)
mod = SeqDataModule(**yml["data"])
eugene = FCN(**yml["model"])

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [9]:
eugene

FCN(
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=264, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.2, inplace=False)
      (3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Linear(in_features=128, out_features=64, bias=True)
      (5): ReLU(inplace=True)
      (6): Dropout(p=0.2, inplace=False)
      (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [10]:
mod.seq_file

'/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/ohe_seq/0.09-0.4_X-training_ohe-seq.npy'

# Train the model with a trainer

In [11]:
LOG_DIR = f"../test/test_logs/config"
logger = TensorBoardLogger(LOG_DIR, name=MODEL, version=f"{STRAND}_{TASK}")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [12]:
trainer.fit(model=eugene, datamodule=mod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name     | Type                      | Params
-------------------------------------------------------
0 | fcn      | BasicFullyConnectedModule | 42.6 K
1 | accuracy | Accuracy                  | 0     
2 | auroc    | AUROC                     | 0     
-------------------------------------------------------
42.6 K    Trainable params
0         Non-trainable params
42.6 K    Total params
0.170     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Scratch